In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, min

In [7]:
spark = SparkSession.builder.appName("airbnbSparkonHDFS").getOrCreate()

In [8]:
hdfs_path = "hdfs://localhost:9000/user/hadoop/"
file_path = hdfs_path + "airbnb.csv"
df_air = spark.read.csv(path=file_path, header=True, inferSchema=True)
df_air = df_air.fillna({"overall_satisfaction": 3.0})

In [9]:
df_air = df_air.groupby("neighborhood").agg(
    avg("reviews").alias("avg of reviews"),
    avg("overall_satisfaction").alias("avg of overall_satisfaction"),
    avg("price").alias("avg of price"),
    max("reviews").alias("max of reviews"),
    min("reviews").alias("min of reviews"),
    max("price").alias("max of price"),
    min("price").alias("min of price"),
).orderBy("neighborhood")
df_air.show()

+--------------------+------------------+---------------------------+------------------+--------------+--------------+------------+------------+
|        neighborhood|    avg of reviews|avg of overall_satisfaction|      avg of price|max of reviews|min of reviews|max of price|min of price|
+--------------------+------------------+---------------------------+------------------+--------------+--------------+------------+------------+
|             Allston| 10.31400966183575|          3.710144927536232|100.01932367149759|           118|             0|       550.0|        20.0|
|            Back Bay|11.296819787985866|         3.9558303886925796|237.51590106007066|           283|             0|      1372.0|        41.0|
|         Bay Village|10.947368421052632|         3.9473684210526314| 254.1578947368421|            47|             0|      1200.0|        81.0|
|         Beacon Hill|              17.8|          4.087804878048781|215.90731707317073|           214|             0|       849.0

In [10]:
df_air.write.csv(hdfs_path + 'sorted_neighborhood_factors.csv', header= True)